In [1]:
from sklearn.linear_model import LinearRegression
from itertools import product
import pandas as pd
import numpy as np
import matplotlib.style
import matplotlib.pyplot as plt
import yfinance as yf

pd.options.display.float_format = '{:.6f}'.format
plt.style.use("seaborn-v0_8")
from itertools import product

In [38]:
symbol = "EURUSD=X"
start = "2023-01-01"
end = "2024-06-01"
interval = '1h'
lags=5

In [39]:
df = yf.download(symbol, start, end, interval=interval).Close.to_frame()
df.rename(columns={"Close": "price"}, inplace=True)
df['returns'] = np.log(df['price'].div(df['price'].shift(1)))
df

[*********************100%%**********************]  1 of 1 completed


,price,returns
Datetime,,
2023-01-02 00:00:00+00:00,1.070893,NaN
2023-01-02 01:00:00+00:00,1.070664,-0.000214
2023-01-02 02:00:00+00:00,1.070435,-0.000214
2023-01-02 03:00:00+00:00,1.070893,0.000428
2023-01-02 04:00:00+00:00,1.070091,-0.000749
...,...,...
2024-05-31 18:00:00+01:00,1.084952,-0.000108
2024-05-31 19:00:00+01:00,1.084481,-0.000434
2024-05-31 20:00:00+01:00,1.085305,0.000759


In [40]:
cols = []
for lag in range(1, lags+1):
    col = "lag{}".format(lag)
    df[col] = df.returns.shift(lag)
    cols.append(col)
df.dropna(inplace=True)
df

,price,returns,lag1,lag2,lag3,lag4,lag5
Datetime,,,,,,,
2023-01-02 06:00:00+00:00,1.068719,-0.000535,-0.000749,-0.000749,0.000428,-0.000214,-0.000214
2023-01-02 07:00:00+00:00,1.068148,-0.000534,-0.000535,-0.000749,-0.000749,0.000428,-0.000214
2023-01-02 08:00:00+00:00,1.068262,0.000107,-0.000534,-0.000535,-0.000749,-0.000749,0.000428
2023-01-02 09:00:00+00:00,1.068148,-0.000107,0.000107,-0.000534,-0.000535,-0.000749,-0.000749
2023-01-02 10:00:00+00:00,1.068376,0.000214,-0.000107,0.000107,-0.000534,-0.000535,-0.000749
...,...,...,...,...,...,...,...
2024-05-31 18:00:00+01:00,1.084952,-0.000108,0.000000,-0.000977,-0.000869,-0.001197,0.002609
2024-05-31 19:00:00+01:00,1.084481,-0.000434,-0.000108,0.000000,-0.000977,-0.000869,-0.001197
2024-05-31 20:00:00+01:00,1.085305,0.000759,-0.000434,-0.000108,0.000000,-0.000977,-0.000869


In [41]:
cols

['lag1', 'lag2', 'lag3', 'lag4', 'lag5']

In [42]:
lm = LinearRegression(fit_intercept=True)
lm.fit(df[cols], df['returns'])

LinearRegression()

In [43]:
lm.coef_

array([-0.00783276,  0.01552693,  0.0112229 ,  0.01698412, -0.00223356])

In [44]:
lm.intercept_

np.float64(1.6578877743375644e-06)

In [45]:
df['pred'] = lm.predict(df[cols].values)

C:\code\trading\v1\pythonProject\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [46]:
df.pred = np.sign(df.pred)
df

,price,returns,lag1,lag2,lag3,lag4,lag5,pred
Datetime,,,,,,,,
2023-01-02 06:00:00+00:00,1.068719,-0.000535,-0.000749,-0.000749,0.000428,-0.000214,-0.000214,-1.000000
2023-01-02 07:00:00+00:00,1.068148,-0.000534,-0.000535,-0.000749,-0.000749,0.000428,-0.000214,-1.000000
2023-01-02 08:00:00+00:00,1.068262,0.000107,-0.000534,-0.000535,-0.000749,-0.000749,0.000428,-1.000000
2023-01-02 09:00:00+00:00,1.068148,-0.000107,0.000107,-0.000534,-0.000535,-0.000749,-0.000749,-1.000000
2023-01-02 10:00:00+00:00,1.068376,0.000214,-0.000107,0.000107,-0.000534,-0.000535,-0.000749,-1.000000
...,...,...,...,...,...,...,...,...
2024-05-31 18:00:00+01:00,1.084952,-0.000108,0.000000,-0.000977,-0.000869,-0.001197,0.002609,-1.000000
2024-05-31 19:00:00+01:00,1.084481,-0.000434,-0.000108,0.000000,-0.000977,-0.000869,-0.001197,-1.000000
2024-05-31 20:00:00+01:00,1.085305,0.000759,-0.000434,-0.000108,0.000000,-0.000977,-0.000869,-1.000000


In [47]:
df.pred.value_counts()

pred
1.000000     4809
-1.000000    3958
Name: count, dtype: int64

In [48]:
hits = np.sign(df.returns*df.pred).value_counts()
hits

1.000000     4063
-1.000000    3970
0.000000      734
Name: count, dtype: int64

In [49]:
hit_ratio = hits[1.0] / sum(hits)
hit_ratio

np.float64(0.46344245465951867)